In [22]:
# Using glob to merge our costar export data to get around 499 Record Limit
import pandas as pd
import glob

# Get a list of all CSV files in the current directory
csv_files = glob.glob('*.csv')

# Loop and read each CSV and add it to a list of DataFrames
dfs = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        dfs.append(df)
        print(f"✅ Loaded {file} with shape {df.shape}")
    except Exception as e:
        print(f"❌ Failed to read {file}: {e}")

# Combine all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

# Optional: Save the result to a new CSV file
combined_df.to_csv('All_Costar_MFG_Exports_Combined.csv', index=False)

print(f"\n✅ Combined {len(csv_files)} files into a DataFrame with {combined_df.shape[0]} rows and {combined_df.shape[1]} columns.")



✅ Loaded MFG_1_To_8999_SF_CSV.csv with shape (487, 51)
✅ Loaded MFG_9000_To_19000_SF_CSV.csv with shape (490, 51)
✅ Loaded MFG_40k_To_140K_SF_CSV.csv with shape (496, 51)
✅ Loaded MFG_140k_To_2Milli_SF_CSV.csv with shape (170, 51)
✅ Loaded MFG_19000_To_40000_SF_CSV.csv with shape (491, 51)

✅ Combined 5 files into a DataFrame with 2134 rows and 51 columns.


In [ ]:
print(list(combined_df.columns))


In [ ]:
''' from simple_salesforce import Salesforce
# Replace these placeholders with your Salesforce credentials
USERNAME = 'jgilmour@lee-associates.com'  # Your Salesforce username
PASSWORD = 'Jazzy777.'  # Your Salesforce password
SECURITY_TOKEN = 'K7J31vLvNcoImesGFm1iqXc84'

# Replace these with your credentials
sf = Salesforce(USERNAME,PASSWORD,SECURITY_TOKEN,domain='login')

# Test query
query = """
SELECT Sub_Market_Name__c, ascendix__Property__r.Name, Property_Record_Type__c, 
Property_Sub_Type__c, Total_Land_Area_SF__c, ascendix__ListingPrice__c,
Listing_Price_PSF__c, ascendix__ListingBrokerContact__r.Name, 
ascendix__ListingBrokerCompany__r.Name, ascendix__SaleDate__c, ascendix__Property__Costar_ID__c,
FROM ascendix__Property__c
"""

#Execute into DF
results = sf.query(query)

print(results)
'''

In [ ]:
''' # Get metadata for all accessible objects, checking our user permissions
all_objects = sf.describe()['sobjects']

# Filter for objects with "Property" in their name or label
property_objects = [
    (obj['name'], obj['label']) 
    for obj in all_objects 
    if 'property' in obj['name'].lower() or 'property' in obj['label'].lower()
]

# Display the results
for api_name, label in property_objects:
    print(f"{label} → {api_name}")
'''

In [32]:
# Reading both exports with pandas and loading into csv so that we can merge on matching (PK) --> costar Id's



import pandas as pd

# Load Ascendix data
ascendix_df = pd.read_csv(r"Ascendix_Where_Costar__ID_Not_Blank.csv")

# Load Costar data
costar_df = pd.read_csv(r"All_Costar_MFG_Exports_Combined.csv")

# Looking at df's
ascendix_df.head()
costar_df.head()


# By default, Ascendix & Costar will normalize data differently ---> need to standardize
costar_df["PropertyID"] = costar_df["PropertyID"].astype(str)
ascendix_df["PropertyID"] = ascendix_df["PropertyID"].astype(str)



In [ ]:
# Checking that our df has proper column headers

with pd.option_context('display.max_columns', None):
    print(costar_df.head())  # or df.head(100) if you want more rows



In [ ]:
# Need to Standardize column names on PK C_ID

ascendix_df.rename(columns={"Costar": "PropertyID"}, inplace=True)
print(ascendix_df)
# Done

In [ ]:
# Merge datasets on Costar ID
# I want to isolate only the Costar Properties that are NOT found in Ascendix 
merged_df = costar_df.merge(ascendix_df, on="PropertyID", how="left", indicator=True)

# Identify missing properties (PropertyID's that exist in Costar and not in Ascendix ---> 'left_only'
missing_properties = merged_df[merged_df["_merge"] == "left_only"]

# Save missing properties to a NEW CSV so that they are all in one place for further investigation 
missing_properties.to_csv("missing_properties.csv", index=False)

# List all column names
print(missing_properties.columns)
# Print the number of rows in missing_properties
print(f"Number of rows: {len(missing_properties)}")
# Summary of the DataFrame
missing_properties.info()
# Print the shape of the DataFrame
print(f"Shape of missing_properties: {missing_properties.shape}")


In [ ]:
# Move PK to front of DF to make more workable

# Get the list of columns
cols = list(missing_properties.columns)

# Move 'PropertyID' to be after 'PropertyAddress'
cols.remove("PropertyID")  # Remove it from its current position
index = cols.index("Property Address") + 1  # Find position after 'PropertyAddress'
cols.insert(index, "PropertyID")  # Insert it there

# Reorder the DataFrame
missing_properties = missing_properties[cols]

# Double check change took effect

print(missing_properties)
missing_properties.to_csv("missing_properties_Final_V.csv", index=False)
'''missing_properties.to_csv("missing_properties.csv", index=False)'''



In [ ]:
duplicates = missing_properties[missing_properties.duplicated(subset=["PropertyID"], keep=False)]
print(f"Shape of dupes: {duplicates.shape}")
print(duplicates)


In [12]:
# Hard coding ID's where crane > 1 as MFG into own column
import pandas as pd
df = pd.read_csv("MFG_missing_properties_in_Final_V.csv")
print("Completed Read")
print(missing_properties)



Completed Read
               Property Address  PropertyID Property Name_x Property Type  \
0                  5713 13th St    19686100      Building A          Flex   
1                 201 6th St. N    12290276             NaN          Flex   
2             2418 N Frazier St    12719672    Building 107          Flex   
3               1904 Hialeah Dr     5785312             NaN          Flex   
4                104 E Motel Dr    10298615             NaN          Flex   
..                          ...         ...             ...           ...   
533          1502 Fort Worth St    20590008             NaN    Industrial   
534          1502 Fort Worth St    20590006      Building 1    Industrial   
535  12221 N Houston Rosslyn Rd    12632772      Building C    Industrial   
536              14888 Kirby Dr    20296859             NaN    Industrial   
537              8450 Rayson Rd    20817727             NaN    Industrial   

    Building Class Building Status    RBA  Total Available S

In [13]:
# Datatype check

print(df.dtypes)


Property Address                  object
PropertyID                         int64
Property Name_x                   object
Property Type                     object
Building Class                    object
Building Status                   object
RBA                                int64
Total Available Space (SF)       float64
Secondary Type                    object
Market Name                       object
Submarket Name                    object
Leasing Company Name              object
Leasing Company Contact           object
Submarket Cluster                 object
City                              object
State                             object
Zip                               object
County Name                       object
Sale Company Name                 object
Sale Company Contact              object
For Sale Price                   float64
For Sale Status                   object
Last Sale Date                    object
Last Sale Price                   object
Percent Leased  

In [18]:
import numpy as np
df['missing_properties'] = np.where(df['Number Of Cranes'] >= 1, 'Yes', 'No') 
print("Completed")

TypeError: '>=' not supported between instances of 'str' and 'int'

In [15]:
print(df.columns.tolist())


['Property Address', 'PropertyID', 'Property Name_x', 'Property Type', 'Building Class', 'Building Status', 'RBA', 'Total Available Space (SF)', 'Secondary Type', 'Market Name', 'Submarket Name', 'Leasing Company Name', 'Leasing Company Contact', 'Submarket Cluster', 'City', 'State', 'Zip', 'County Name', 'Sale Company Name', 'Sale Company Contact', 'For Sale Price', 'For Sale Status', 'Last Sale Date', 'Last Sale Price', 'Percent Leased', 'Year Built', 'Year Renovated', 'Parking Ratio', 'Tenancy', 'Floodplain Area', 'Ceiling Ht', 'Column Spacing', 'Number Of Cranes', 'Number Of Loading Docks', 'Drive Ins', 'Power', 'Rail Lines', 'Sewer', 'Building Park', 'Construction Material', 'Direct Available Space', 'Direct Vacant Space', 'Flood Risk Area', 'Land Area (AC)', 'Latitude', 'Max Building Contiguous Space', 'Office Space', 'Owner Contact', 'Tax Year', 'Vacancy %', 'Longitude', 'Property Name_y', '_merge']


In [ ]:
 print(missing_properties)

In [ ]:
''' 1011 Costar ID Blank, 7166 Costar ID NOT Blank ---> Final Csv After Cleaning = 538 MFG Properties

    538 

IF crane > 1 we hardcode as mfg no matter what    ''' 

In [ ]:
''''
import pandas as pd
from simple_salesforce import Salesforce

# Authenticate
USERNAME = 'jgilmour@lee-associates.com'  # Your Salesforce username
PASSWORD = 'Nalanala7777.'  # Your Salesforce password
SECURITY_TOKEN = 'SQhuuEAv1e3jjUy0oS5JPUI8h'
sf = Salesforce(username=USERNAME, password=PASSWORD, security_token=SECURITY_TOKEN)



# First we need to load the missing_properties.csv into a Pandas DataFrame
# missing_properties.csv MUST have headers that align with Salesforce field names
# Batches?
missing_properties_df = pd.read_csv("missing_properties.csv")

# Iterate through the DataFrame and add each property
for index, row in missing_properties_df.iterrows():
    try:
        property_data = {
            "Record_Type__c": row["Record Type"],  # EXAMPLE FIELD MAPPINGS, NEED TO GET ALL Squared away before pushing fr fr
            "Property_Sub_Type__c": row["Property Sub Type"], # For every field that needs to be populated in Ascendix, find the Salesforce API field name and pair it with the matching header name from Cleaned CSV
            "Property_Name__c": row["Property Name"],
            "Location_Description__c": row["Location Description"],
            "Street__c": row["Street"],
            "City__c": row["City"],
            "State_Province__c": row["State/Province"],
            "Zip_Postal_Code__c": row["Zip/Postal Code"],
            "County__c": row["County"],
            "Geolocation_Coordinates__c": row["Geolocation Coordinates"],
            "Costar_ID__c": row["Costar ID"],
            "Tenancy__c": row["Tenancy"],
            "Building_Status__c": row["Building Status"],
            "Submarket__c": row["Submarket"],
            "Market__c": row["Market"],
            "Region__c": row["Region"],
            "Total_Building_Area_SF__c": row["Total Building Area (SF)"],
            "Office_Area_SF__c": row["Office Area (SF)"],
            "Total_Land_Area_Acres__c": row["Total Land Area (Acres)"],
            "Loading_Type__c": row["Loading Type"],
            "Docks__c": row["Docks"],
            "Drive_Ins__c": row["Drive Ins"],
            "Parking_Total__c": row["Parking (Total)"],
            "Listing_Broker_Company__c": row["Listing Broker Company"],
            "Listing_Broker_Contact__c": row["Listing Broker Contact"],
        }
            #Salesforce_API_names           #Header Names from CSV We cleaned from Costar on Right Side
        # Add property to Ascendix
        sf.Property__c.create(property_data)    #create method of the simple_salesforce library to add records programmatically.
        print(f"Successfully added property: {row['Property Name']}")
    except Exception as e:
        print(f"Error adding property: {row['Property Name']} - {e}")


# Need to add some sort of logging logic to handle failed inserts and make troubleshooting more straightforward
''''